In [21]:
import pandas as pd
import random
from pyspark.sql import SparkSession

StatementMeta(, 5a9f9e5c-0db5-4b38-a772-ba123e3e1d2e, 23, Finished, Available)

In [18]:
def add_internal_values():
    df_in = spark.read.format("delta").load("Tables/internalfile")
    df_in = df_in.toPandas()   
    df_in = df_in.drop_duplicates(subset='cust_id', keep="first")
    df_loan = spark.read.format("delta").load("Tables/loan_master")
    df_loan = df_loan.toPandas()
    df_loan = df_loan.drop_duplicates(subset='cust_id', keep="first")
    df_write = pd.DataFrame(columns=['cust_id', 'alert_impact', 'alert_type','alert_attribute','alert_date',
                                 'alert_tat','Date_action_taken','alert_status','alert_assgined_to','alert_resolved_by','account_type','industry_type'])

    #merge both the dataframes
    # df_in = df_in['cust_id'].astype(int)
    # df_loan = df_loan['cust_id'].astype(int)
    df = df_in.merge(df_loan, on="cust_id", how = 'inner')
    names_values={
        'Auditor_Report_Discrepancies':16,
        'Internal_Fraud_Alert':17,
        'Internal_Dispute':18,
        'Change_in_Statutory_Auditors':19,
        'Resignation_Key_Personnel':20,
        'Project_Sponsor_Withdrawal':21,
        'Financial_Results_Announcement':22,
        'Utility_Disruption':23
                
             }

    for i in list(names_values.keys()):
        df_high = df[['cust_id',i,'Renewal_Due_Date']].copy()
        df_high.rename(columns = {i:'alert_impact'}, inplace = True)
        df_high['alert_impact'] = df_high['alert_impact'].map({'High':3,'Medium':2,'Low':1,'No Change':0})
        df_high['alert_impact'] = df_high['alert_impact'].astype(int)
        df_high['alert_type']=4
        df_high['alert_attribute']=names_values[i]
        df_high['alert_date']=pd.to_datetime(df_high['Renewal_Due_Date'],format='mixed')+pd.DateOffset(days=random.choice([1, 4, 8, 10, 3,5,7,8]))
        df_high['alert_tat']= [random.randint(1,60) for _ in range(len(df_high))]
        df_high['Date_action_taken']=pd.to_datetime(df_high['alert_date'])+pd.to_timedelta(df_high['alert_tat'],unit='D' )
        open_prob=0.7
        df_high['alert_status']=['open' if random.random()<open_prob else 'Closed' for _ in range(len(df_high))]
        df_high['alert_assgined_to']= [random.randint(1,20) for _ in range(len(df_high))]
        df_high['alert_resolved_by']= [random.randint(1,20) for _ in range(len(df_high))]
        df_high['account_type']=0
        df_high['industry_type']= 0
        df_high=df_high.drop(['Renewal_Due_Date'], axis=1)
        df_high = df_high.drop_duplicates()
        df_write = pd.concat([df_write, df_high])
    return df_write


StatementMeta(, 5a9f9e5c-0db5-4b38-a772-ba123e3e1d2e, 20, Finished, Available)

In [23]:
df_write=add_internal_values()
df_write=spark.createDataFrame(df_write)
df_write.write.format("delta").mode("overwrite").save("Tables/alert_file_5")


StatementMeta(, 5a9f9e5c-0db5-4b38-a772-ba123e3e1d2e, 25, Finished, Available)